In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score
import sklearn.impute
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler


import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

import acquire
import prepare
import split_scale

# Classification Modeling Exercise

In this exercise, we'll continue working with the titanic dataset and building logistic regression models. Throughout this exercise, be sure you are training, evaluation, and comparing models on the train and validate datasets. The test dataset should only be used for your final model.

For all of the models you create, choose a threshold that optimizes for accuracy.

Do your work for these exercises in either a notebook or a python script named model within your classification directory.

1. Create another model that includes age in addition to fare and pclass. Does this model perform better than your previous one?
2. Include sex in your model as well. Note that you'll need to encode this feature before including it in a model.
3. Try out other combinations of features and models.
4. Choose you best model and evaluate it on the test dataset. Is it overfit?
5. **Bonus**: How do different strategies for handling the missing values in the age column affect model performance?
6. **Bonus**: How do different strategies for encoding sex affect model performance?
7. **Bonus**: scikit-learn's LogisticRegression classifier is actually applying a regularization penalty to the coefficients by default. This penalty causes the magnitude of the coefficients in the resulting model to be smaller than they otherwise would be. This value can be modified with the C hyper parameter. Small values of C correspond to a larger penalty, and large values of C correspond to a smaller penalty.

Try out the following values for C and note how the coefficients and the model's performance on both the dataset it was trained on and on the validate split are affected.

$C=.01,.1,1,10,100,1000$

- 8. **Bonus Bonus**: how does scaling the data interact with your choice of C?

In [2]:
df = acquire.get_titanic_data()

In [3]:
df = df.drop(columns="deck")

In [4]:
# fix embarktown and embarked

df.embark_town = df.embark_town.fillna('Southampton')
df.embarked = df.embarked.fillna('S')

In [5]:
df.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,Southampton,1


### Initial Model

In [6]:
train, test = train_test_split(df, random_state=123, train_size=.8)
train, validate = train_test_split(train, train_size=.8)

X = train[['pclass', 'fare']]
y = train[['survived']]

logit1 = LogisticRegression(random_state=123).fit(X, y)

In [7]:
logit1.predict(X)

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [8]:
logit1.predict_proba(X)

array([[0.39270404, 0.60729596],
       [0.36923441, 0.63076559],
       [0.73300777, 0.26699223],
       ...,
       [0.56304853, 0.43695147],
       [0.73309925, 0.26690075],
       [0.57591006, 0.42408994]])

In [9]:
logit1.score(X, y)

0.680140597539543

### 1. Create another model that includes age in addition to fare and pclass. Does this model perform better than your previous one?

In [10]:
train.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,embark_town,alone
662,662,0,1,male,47.0,0,0,25.5875,S,First,Southampton,1
137,137,0,1,male,37.0,1,0,53.1000,S,First,Southampton,0
840,840,0,3,male,20.0,0,0,7.9250,S,Third,Southampton,1
142,142,1,3,female,24.0,1,0,15.8500,S,Third,Southampton,0
444,444,1,3,male,NaN,0,0,8.1125,S,Third,Southampton,1


In [11]:
imputer = sklearn.impute.SimpleImputer(strategy='mean')
imputer.fit(train[['age']])
train.age = imputer.transform(train[['age']])
validate.age = imputer.transform(validate[['age']])
test.age = imputer.transform(test[['age']])

In [12]:
train.isna().sum()

passenger_id    0
survived        0
pclass          0
sex             0
age             0
sibsp           0
parch           0
fare            0
embarked        0
class           0
embark_town     0
alone           0
dtype: int64

In [13]:
print('    test: %d rows x %d columns' % test.shape)
print('   train: %d rows x %d columns' % train.shape)
print('validate: %d rows x %d columns' % validate.shape)

    test: 179 rows x 12 columns
   train: 569 rows x 12 columns
validate: 143 rows x 12 columns


In [14]:
X = train[['pclass', 'fare', 'age']]
y = train[['survived']]

logit2 = LogisticRegression(random_state=123).fit(X, y)

In [15]:
logit2.predict(X)

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [16]:
logit2.predict_proba(X)

array([[0.47820141, 0.52179859],
       [0.38570517, 0.61429483],
       [0.70073072, 0.29926928],
       ...,
       [0.62434434, 0.37565566],
       [0.71267187, 0.28732813],
       [0.52477345, 0.47522655]])

In [17]:
logit2.score(X, y)

0.6906854130052724

In [18]:
X_validate = validate[['pclass', 'fare', 'age']]
y_validate = validate[['survived']]

In [19]:
logit2.score(X_validate, y_validate)

0.7272727272727273

### 2. Include sex in your model as well. Note that you'll need to encode this feature before including it in a model.

In [20]:
train.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,embark_town,alone
662,662,0,1,male,47.000000,0,0,25.5875,S,First,Southampton,1
137,137,0,1,male,37.000000,1,0,53.1000,S,First,Southampton,0
840,840,0,3,male,20.000000,0,0,7.9250,S,Third,Southampton,1
142,142,1,3,female,24.000000,1,0,15.8500,S,Third,Southampton,0
444,444,1,3,male,29.001305,0,0,8.1125,S,Third,Southampton,1


In [21]:
# encode sex for both train and test
le = LabelEncoder()
train['sex_encoder'] = le.fit_transform(train.sex)
validate['sex_encoder'] = le.fit_transform(validate.sex)
test['sex_encoder'] = le.transform(test.sex)

In [22]:
train.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,embark_town,alone,sex_encoder
662,662,0,1,male,47.000000,0,0,25.5875,S,First,Southampton,1,1
137,137,0,1,male,37.000000,1,0,53.1000,S,First,Southampton,0,1
840,840,0,3,male,20.000000,0,0,7.9250,S,Third,Southampton,1,1
142,142,1,3,female,24.000000,1,0,15.8500,S,Third,Southampton,0,0
444,444,1,3,male,29.001305,0,0,8.1125,S,Third,Southampton,1,1


In [23]:
print('    test: %d rows x %d columns' % test.shape)
print('   train: %d rows x %d columns' % train.shape)
print('validate: %d rows x %d columns' % validate.shape)

    test: 179 rows x 13 columns
   train: 569 rows x 13 columns
validate: 143 rows x 13 columns


In [24]:
X = train[['pclass', 'fare', 'age', 'sex_encoder']]
y = train[['survived']]

logit3 = LogisticRegression(random_state=123).fit(X, y)

In [25]:
logit3.predict(X)

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,

In [26]:
logit3.predict_proba(X)

array([[0.64411205, 0.35588795],
       [0.59407556, 0.40592444],
       [0.8565467 , 0.1434533 ],
       ...,
       [0.78071714, 0.21928286],
       [0.86071173, 0.13928827],
       [0.73476762, 0.26523238]])

In [27]:
logit3.score(X, y)

0.7785588752196837

In [28]:
X_validate = validate[['pclass', 'fare', 'age', 'sex_encoder']]
y_validate = validate[['survived']]

logit3.score(X_validate, y_validate)

0.8111888111888111

### 3. Try out other combinations of features and models.

In [29]:
train.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,embark_town,alone,sex_encoder
662,662,0,1,male,47.000000,0,0,25.5875,S,First,Southampton,1,1
137,137,0,1,male,37.000000,1,0,53.1000,S,First,Southampton,0,1
840,840,0,3,male,20.000000,0,0,7.9250,S,Third,Southampton,1,1
142,142,1,3,female,24.000000,1,0,15.8500,S,Third,Southampton,0,0
444,444,1,3,male,29.001305,0,0,8.1125,S,Third,Southampton,1,1


In [30]:
# encoding embarked

encoder = sklearn.preprocessing.OneHotEncoder()
encoder.fit(train[["embarked"]])

m1 = encoder.transform(train[["embarked"]]).todense()

train = pd.concat([train, pd.DataFrame(m1, columns=encoder.categories_[0], index=train.index)], axis=1)

m2 = encoder.transform(validate[["embarked"]]).todense()

validate = pd.concat([validate, pd.DataFrame(m2, columns=encoder.categories_[0], index=validate.index)], axis=1)

m3 = encoder.transform(test[["embarked"]]).todense()

test = pd.concat([test, pd.DataFrame(m3, columns=encoder.categories_[0], index=test.index)], axis=1)

In [31]:
train.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,embark_town,alone,sex_encoder,C,Q,S
662,662,0,1,male,47.000000,0,0,25.5875,S,First,Southampton,1,1,0.0,0.0,1.0
137,137,0,1,male,37.000000,1,0,53.1000,S,First,Southampton,0,1,0.0,0.0,1.0
840,840,0,3,male,20.000000,0,0,7.9250,S,Third,Southampton,1,1,0.0,0.0,1.0
142,142,1,3,female,24.000000,1,0,15.8500,S,Third,Southampton,0,0,0.0,0.0,1.0
444,444,1,3,male,29.001305,0,0,8.1125,S,Third,Southampton,1,1,0.0,0.0,1.0


In [32]:
print('    test: %d rows x %d columns' % test.shape)
print('   train: %d rows x %d columns' % train.shape)
print('validate: %d rows x %d columns' % validate.shape)

    test: 179 rows x 16 columns
   train: 569 rows x 16 columns
validate: 143 rows x 16 columns


In [33]:
X = train[['pclass', 'fare', 'age', 'sex_encoder', 'C', 'Q', 'S']]
y = train[['survived']]

logit4 = LogisticRegression(random_state=123).fit(X, y)

In [34]:
logit4.predict(X)

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,

In [35]:
logit4.predict_proba(X)

array([[0.66028144, 0.33971856],
       [0.61503   , 0.38497   ],
       [0.86746432, 0.13253568],
       ...,
       [0.79721863, 0.20278137],
       [0.87164984, 0.12835016],
       [0.74657735, 0.25342265]])

In [36]:
logit4.score(X, y)

0.7855887521968365

In [37]:
X_validate = validate[['pclass', 'fare', 'age', 'sex_encoder', 'C', 'Q', 'S']]
y_validate = validate[['survived']]

logit4.score(X_validate, y_validate)

0.8111888111888111

### 4. Choose you best model and evaluate it on the test dataset. Is it overfit?

In [38]:
X_test = test[['pclass', 'fare']]
y_test = test[['survived']]

logit1.score(X_test, y_test)

0.7039106145251397

In [39]:
X_test = test[['pclass', 'fare', 'age']]
y_test = test[['survived']]

logit2.score(X_test, y_test)

0.7318435754189944

In [40]:
X_test = test[['pclass', 'fare', 'age', 'sex_encoder']]
y_test = test[['survived']]

logit3.score(X_test, y_test)

0.7988826815642458

In [41]:
X_test = test[['pclass', 'fare', 'age', 'sex_encoder', 'C', 'Q', 'S']]
y_test = test[['survived']]

logit4.score(X_test, y_test)

0.8156424581005587

#### Looks like model 4 is the best fit without overfitting for the train set.

### 5. **Bonus**: How do different strategies for handling the missing values in the age column affect model performance?

By choosing to drop all the NA's caused major issues within the models. By imputing values the models were able to run without issue.

### 6. **Bonus**: How do different strategies for encoding sex affect model performance?

Because the choice is binary, the difference between One-Hot and Label encoding will result in similar findings.

### 7. **Bonus**: scikit-learn's LogisticRegression classifier is actually applying a regularization penalty to the coefficients by default. This penalty causes the magnitude of the coefficients in the resulting model to be smaller than they otherwise would be. This value can be modified with the C hyper parameter. Small values of C correspond to a larger penalty, and large values of C correspond to a smaller penalty.

> Try out the following values for C and note how the coefficients and the model's performance on both the dataset it was trained on and on the validate split are affected.

> $C=.01,.1,1,10,100,1000$

In [42]:
X = train[['pclass', 'fare', 'age', 'sex_encoder', 'C', 'Q', 'S']]
y = train[['survived']]

logit5 = LogisticRegression(random_state=123, C=.01).fit(X, y)

In [43]:
logit5.score(X, y)

0.6959578207381371

In [44]:
X_validate = validate[['pclass', 'fare', 'age', 'sex_encoder', 'C', 'Q', 'S']]
y_validate = validate[['survived']]

logit5.score(X_validate, y_validate)

0.7342657342657343

In [45]:
X = train[['pclass', 'fare', 'age', 'sex_encoder', 'C', 'Q', 'S']]
y = train[['survived']]

logit6 = LogisticRegression(random_state=123, C=.1).fit(X, y)
logit6.score(X, y)

0.7697715289982425

In [46]:
X_validate = validate[['pclass', 'fare', 'age', 'sex_encoder', 'C', 'Q', 'S']]
y_validate = validate[['survived']]

logit6.score(X_validate, y_validate)

0.8041958041958042

In [47]:
X = train[['pclass', 'fare', 'age', 'sex_encoder', 'C', 'Q', 'S']]
y = train[['survived']]

logit7 = LogisticRegression(random_state=123, C=1.0).fit(X, y)
logit7.score(X, y)

0.7855887521968365

In [48]:
X_validate = validate[['pclass', 'fare', 'age', 'sex_encoder', 'C', 'Q', 'S']]
y_validate = validate[['survived']]

logit7.score(X_validate, y_validate)

0.8111888111888111

In [49]:
X = train[['pclass', 'fare', 'age', 'sex_encoder', 'C', 'Q', 'S']]
y = train[['survived']]

logit8 = LogisticRegression(random_state=123, C=10.0).fit(X, y)
logit8.score(X, y)

0.7803163444639719

In [50]:
X_validate = validate[['pclass', 'fare', 'age', 'sex_encoder', 'C', 'Q', 'S']]
y_validate = validate[['survived']]

logit8.score(X_validate, y_validate)

0.7972027972027972

In [51]:
X = train[['pclass', 'fare', 'age', 'sex_encoder', 'C', 'Q', 'S']]
y = train[['survived']]

logit9 = LogisticRegression(random_state=123, C=100.0).fit(X, y)
logit9.score(X, y)

0.7785588752196837

In [52]:
X_validate = validate[['pclass', 'fare', 'age', 'sex_encoder', 'C', 'Q', 'S']]
y_validate = validate[['survived']]

logit9.score(X_validate, y_validate)

0.8041958041958042

In [53]:
X = train[['pclass', 'fare', 'age', 'sex_encoder', 'C', 'Q', 'S']]
y = train[['survived']]

logit10 = LogisticRegression(random_state=123, C=1000.0).fit(X, y)
logit10.score(X, y)

0.7785588752196837

In [54]:
X_validate = validate[['pclass', 'fare', 'age', 'sex_encoder', 'C', 'Q', 'S']]
y_validate = validate[['survived']]

logit10.score(X_validate, y_validate)

0.8041958041958042

### - 8. **Bonus Bonus**: how does scaling the data interact with your choice of C?

In [55]:
train_scaled = train[['age', 'fare']]
validate_scaled = validate[['age', 'fare']]
scaler, train_scaled, validate_scaled = split_scale.min_max_scaler(train_scaled, validate_scaled)
train_scaled.head()

,age,fare
662,0.585323,0.049943
137,0.459663,0.103644
840,0.246042,0.015469
142,0.296306,0.030937
444,0.359152,0.015835


In [56]:
X = train_scaled[['fare', 'age']]
y = train[['survived']]

logit11 = LogisticRegression(random_state=123, C=.01).fit(X, y)
logit11.score(X, y)

0.6063268892794376

In [57]:
X = train_scaled[['fare', 'age']]
y = train[['survived']]

logit11 = LogisticRegression(random_state=123, C=.10).fit(X, y)
logit11.score(X, y)

0.6098418277680141

In [58]:
X = train_scaled[['fare', 'age']]
y = train[['survived']]

logit11 = LogisticRegression(random_state=123, C=1.0).fit(X, y)
logit11.score(X, y)

0.6256590509666081

In [59]:
X = train_scaled[['fare', 'age']]
y = train[['survived']]

logit11 = LogisticRegression(random_state=123, C=10.0).fit(X, y)
logit11.score(X, y)

0.6467486818980668

In [60]:
X = train_scaled[['fare', 'age']]
y = train[['survived']]

logit11 = LogisticRegression(random_state=123, C=100.0).fit(X, y)
logit11.score(X, y)

0.6362038664323374

In [61]:
X = train_scaled[['fare', 'age']]
y = train[['survived']]

logit11 = LogisticRegression(random_state=123, C=1000.0).fit(X, y)
logit11.score(X, y)

0.6362038664323374

# Class Logic Regression Breakdown Review Section

# Classification Decision Tree Exercises

1. Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)
1. Evaluate your in-sample results using the model score, confusion matrix, and classification report.
1. Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.
1. Run through steps 2-4 using entropy as your measure of impurity.
1. Which performs better on your in-sample data?

In [62]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [63]:
train.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,embark_town,alone,sex_encoder,C,Q,S
662,662,0,1,male,47.000000,0,0,25.5875,S,First,Southampton,1,1,0.0,0.0,1.0
137,137,0,1,male,37.000000,1,0,53.1000,S,First,Southampton,0,1,0.0,0.0,1.0
840,840,0,3,male,20.000000,0,0,7.9250,S,Third,Southampton,1,1,0.0,0.0,1.0
142,142,1,3,female,24.000000,1,0,15.8500,S,Third,Southampton,0,0,0.0,0.0,1.0
444,444,1,3,male,29.001305,0,0,8.1125,S,Third,Southampton,1,1,0.0,0.0,1.0


In [96]:
X = train[['pclass', 'fare', 'age', 'sex_encoder', 'C', 'Q', 'S']]
y = train[['survived']]

In [123]:
clf = DecisionTreeClassifier(max_depth=7, random_state=123)
clf

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=123, splitter='best')

In [124]:
clf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=123, splitter='best')

In [125]:
y_pred = clf.predict(X)
y_pred_proba = clf.predict_proba(X)

In [126]:
y_pred[0:10]

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

In [127]:
y_pred_proba

array([[1.        , 0.        ],
       [0.61904762, 0.38095238],
       [0.88068182, 0.11931818],
       ...,
       [1.        , 0.        ],
       [0.88068182, 0.11931818],
       [0.91666667, 0.08333333]])

In [128]:
#confusion_matrix = pd.DataFrame(confusion_matrix(y, y_pred))
#confusion_matrix.index.name = 'actual'
confusion_matrix

,0,1
actual,,
0,305,40
1,67,157


In [129]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'.format(clf.score(X, y)))

Accuracy of Decision Tree classifier on training set: 0.88


In [130]:
pd.DataFrame(classification_report(y, y_pred, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.873656,0.898477,0.88225,0.886067,0.883427
recall,0.942029,0.790179,0.88225,0.866104,0.882250
f1-score,0.906555,0.840855,0.88225,0.873705,0.880691
support,345.000000,224.000000,0.88225,569.000000,569.000000


In [131]:
print('Accuracy of Decision Tree classifier on test set: {:.2f}'.format(clf.score(X_validate, y_validate)))

Accuracy of Decision Tree classifier on test set: 0.85


In [121]:
import graphviz
from graphviz import Graph

clf = DecisionTreeClassifier()
clf = clf.fit(X, y)

dot_data = export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data) 

graph.render('titanic_decision_tree', view=True)

'titanic_decision_tree.pdf'

In [132]:
print('Accuracy of Decision Tree classifier on test set: {:.2f}'.format(clf.score(X_test, y_test)))

Accuracy of Decision Tree classifier on test set: 0.80
